# Using the sharing API from Python 

In this example, we use $JUPYTERHUB_API_TOKEN to communicate with the sharing API via Python.

The permissions used here are granted via the `c.Spawner.server_token_scopes` config in jupyterhub_config.py

By using this token, any user who has access to this server has access to sharing permissions.

First, get some useful configuration from the server environment:


In [1]:
import os

hub_api = os.environ["JUPYTERHUB_API_URL"]
token = os.environ["JUPYTERHUB_API_TOKEN"]
username = os.environ["JUPYTERHUB_USER"]
user_server = f"{username}/{os.environ['JUPYTERHUB_SERVER_NAME']}"
hub_host = os.environ["JUPYTERHUB_HOST"]
server_base_url = os.environ["JUPYTERHUB_SERVICE_PREFIX"]

hub_api

'http://127.0.0.1:8081/hub/api'

Create a requests.Session to make jupyterhub API requests with our token

In [2]:
import requests

session = requests.Session()
session.headers = {"Authorization": f"Bearer {token}"}

We can check the permissions our token has with a request to /hub/api/user:

In [3]:
r = session.get(f"{hub_api}/user")
r.json()

{'kind': 'user',
 'last_activity': '2024-01-23T11:43:50.800864Z',
 'groups': [],
 'admin': False,
 'name': 'sharer',
 'servers': {'': {'name': '',
   'full_name': 'sharer/',
   'last_activity': '2024-01-23T11:43:50.800864Z',
   'started': '2024-01-23T11:28:44.948553Z',
   'pending': None,
   'ready': True,
   'stopped': False,
   'url': '/user/sharer/',
   'user_options': {},
   'progress_url': '/hub/api/users/sharer/server/progress'}},
 'session_id': None,
 'scopes': ['access:servers!server=sharer/',
  'delete:servers!server=sharer/',
  'groups:shares!server=sharer/',
  'read:groups:shares!server=sharer/',
  'read:servers!server=sharer/',
  'read:shares!server=sharer/',
  'read:users:activity!user=sharer',
  'read:users:groups!user=sharer',
  'read:users:name!user=sharer',
  'servers!server=sharer/',
  'shares!server=sharer/',
  'users:activity!server=sharer/',
  'users:activity!user=sharer',
  'users:shares!server=sharer/']}

We can see who has access to this server:

In [4]:
shares_url = f"{hub_api}/shares/{user_server}"
share_codes_url = f"{hub_api}/share-codes/{user_server}"
r = session.get(shares_url)
r.json()

{'items': [],
 '_pagination': {'offset': 0, 'limit': 200, 'total': 0, 'next': None}}

and if there are any outstanding codes:

In [5]:
r = session.get(share_codes_url)
r.json()

{'items': [],
 '_pagination': {'offset': 0, 'limit': 200, 'total': 0, 'next': None}}

Next, we can create a code:

In [6]:
r = session.post(share_codes_url)
code_info = r.json()
code_info

{'server': {'user': {'name': 'sharer'},
  'name': '',
  'url': '/user/sharer/',
  'ready': True},
 'scopes': ['access:servers!server=sharer/'],
 'id': 'sc_1',
 'created_at': '2024-01-23T11:46:32.154416Z',
 'expires_at': '2024-01-24T11:46:32.153582Z',
 'exchange_count': 0,
 'last_exchanged_at': None,
 'code': 'gTs7DmIhFu6UG5VQkOzRMk2MN-cdDiQ_RONCnkaq5Cg',
 'accept_url': '/hub/accept-share?code=gTs7DmIhFu6UG5VQkOzRMk2MN-cdDiQ_RONCnkaq5Cg'}

In [7]:
from IPython.display import HTML, display

full_accept_url = f"{hub_host}{code_info['accept_url']}"

display(
    HTML(f"""
    <a href='{full_accept_url}'>share this link to grant access</a>
    """)
)

(in jupyterlab, shift-right-click to copy link)

We can now give this to the shared-with user (i.e. us in another private browsing tab).

After accepting the link, we can see who we've shared with again:

In [8]:
session.get(shares_url).json()

{'items': [{'server': {'user': {'name': 'sharer'},
    'name': '',
    'url': '/user/sharer/',
    'ready': True},
   'scopes': ['access:servers!server=sharer/'],
   'user': {'name': 'shared-with'},
   'group': None,
   'kind': 'user',
   'created_at': '2024-01-23T11:46:43.585455Z'}],
 '_pagination': {'offset': 0, 'limit': 200, 'total': 1, 'next': None}}

The share code can also include a `?next=` url parameter, to enable a link to take users to a specific file or view after accepting the code:

In [9]:
from urllib.parse import urlencode

this_notebook_url = server_base_url + "lab/tree/share-api.ipynb"
this_notebook_accept_url = (
    full_accept_url + "&" + urlencode({"next": this_notebook_url})
)
print(this_notebook_accept_url)

display(
    HTML(f"""
    share <a href='{this_notebook_accept_url}'>this link</a>
    to grant access and direct users to this notebook
    """)
)

/hub/accept-share?code=gTs7DmIhFu6UG5VQkOzRMk2MN-cdDiQ_RONCnkaq5Cg&next=%2Fuser%2Fsharer%2Flab%2Ftree%2Fshare-api.ipynb


## Reviewing and managing access

Listing share codes doesn't reveal the code - if you need to get a code, issue a new sharing code.

But we can see in `exchange_count` whether and how often the code has been used

In [10]:
session.get(share_codes_url).json()

{'items': [{'server': {'user': {'name': 'sharer'},
    'name': '',
    'url': '/user/sharer/',
    'ready': True},
   'scopes': ['access:servers!server=sharer/'],
   'id': 'sc_1',
   'created_at': '2024-01-23T11:46:32.154416Z',
   'expires_at': '2024-01-24T11:46:32.153582Z',
   'exchange_count': 1,
   'last_exchanged_at': '2024-01-23T11:46:43.589701Z'}],
 '_pagination': {'offset': 0, 'limit': 200, 'total': 1, 'next': None}}

we can also revoke the code. Codes can be deleted by code or id

In [11]:
session.delete(share_codes_url + f"?id={code_info['id']}")

<Response [204]>

or if you're done sharing via code, you can delete all sharing codes for a server without looking it up their ids:

In [12]:
session.delete(share_codes_url)

<Response [204]>

scopes and expiration can be customized in the request when creating the share code:

In [13]:
import json

options = {
    "scopes": [
        f"access:servers!server={user_server}",  # access the server (default)
        f"servers!server={user_server}",  # start/stop the server
        f"shares!server={user_server}",  # further share the server with others
    ],
    "expires_in": 3600,  # code expires in one hour
}


session.post(share_codes_url, data=json.dumps(options))

<Response [200]>

In [14]:
r = session.get(shares_url)
r.json()

{'items': [{'server': {'user': {'name': 'sharer'},
    'name': '',
    'url': '/user/sharer/',
    'ready': True},
   'scopes': ['access:servers!server=sharer/'],
   'user': {'name': 'shared-with'},
   'group': None,
   'kind': 'user',
   'created_at': '2024-01-23T11:46:43.585455Z'}],
 '_pagination': {'offset': 0, 'limit': 200, 'total': 1, 'next': None}}

## Revoking permissions

We can revoke specific permssions via a PATCH request
by specifying the user (or group) and one or more scopes:

In [15]:
options = {
    "user": "shared-with",
    "scopes": ['shares!server=sharer/'],
}

session.patch(shares_url, data=json.dumps(options))

<Response [200]>

If scopes are unspecified, all permissions are revoked:

In [16]:
options = {
    "user": "shared-with",
}

session.patch(shares_url, data=json.dumps(options))

<Response [200]>

_All_ shared access can be revoked via a DELETE request to the shares URL:

In [17]:
session.delete(shares_url)

<Response [204]>

and we can see that nobody has shared access anymore

In [18]:
session.get(shares_url).json()

{'items': [],
 '_pagination': {'offset': 0, 'limit': 200, 'total': 0, 'next': None}}